In [2]:
!which pip

~/ondemand/data/sys/dashboard/batch_connect/sys/jupyter_app/jupyter_app/output/2bbad118-5705-47b1-afc0-5e7cc34f12d2/venv/bin/pip


In [3]:
!pip install torch==2.7.1 torchvision==0.22.1 torchaudio==2.7.1 --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
  Using cached https://download.pytorch.org/whl/filelock-3.13.1-py3-none-any.whl.metadata (2.8 kB)
  Using cached https://download.pytorch.org/whl/sympy-1.13.3-py3-none-any.whl.metadata (12 kB)
  Using cached https://download.pytorch.org/whl/networkx-3.3-py3-none-any.whl.metadata (5.1 kB)
  Using cached https://download.pytorch.org/whl/fsspec-2024.6.1-py3-none-any.whl.metadata (11 kB)
  Using cached https://download.pytorch.org/whl/cu118/nvidia_cuda_nvrtc_cu11-11.8.89-py3-none-manylinux1_x86_64.whl (23.2 MB)
  Using cached https://download.pytorch.org/whl/cu118/nvidia_cuda_runtime_cu11-11.8.89-py3-none-manylinux1_x86_64.whl (875 kB)
  Using cached https://download.pytorch.org/whl/cu118/nvidia_cuda_cupti_cu11-11.8.87-py3-none-manylinux1_x86_64.whl (13.1 MB)
  Using cached https://download.pytorch.org/whl/cu118/nvidia_cudnn_cu11-9.1.0.70-py3-none-manylinux2014_x86_64.whl (663.9 MB)
  Using cached https://download.pytorch.org/whl/

In [5]:
!pip install transformers

  Using cached tokenizers-0.22.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
  Using cached safetensors-0.6.2-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.1 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached hf_xet-1.1.10-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.7 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 214.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.3/564.3 kB 17.7 MB/s  0:00:00
Using cached hf_xet-1.1.10-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.2 MB)
Using cached tokenizers-0.22.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.3 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.0/802.0 kB 34.0 MB/s  0:00:00
Using cached safetensors-0.6.2-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (485 kB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7/7 [

In [1]:
import argparse
import json
import os
import re
import sys
from typing import Optional

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer


DEFAULT_MODEL = "llm-jp/llm-jp-3.1-13b-instruct4"

/home/acg17145sv/ondemand/data/sys/dashboard/batch_connect/sys/jupyter_app/jupyter_app/output/2bbad118-5705-47b1-afc0-5e7cc34f12d2/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
tokenizer = AutoTokenizer.from_pretrained(DEFAULT_MODEL)

In [ ]:
!pip install accelerate

In [4]:
model = AutoModelForCausalLM.from_pretrained("llm-jp/llm-jp-3.1-13b-instruct4", device_map="auto", torch_dtype=torch.bfloat16)

`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████| 6/6 [00:10<00:00,  1.81s/it]


In [7]:
dialogue_text = ("B: はい。\nA: はい。\nB: いろんな顔がありますね。\nA: 私は1枚紙があって名前が書いてあります。\nB: はいあっ構え書いてあるの。\nA: 123456789。\nB: はい。\nA: 10人。\nB: 12345678910人いますね。\nA: あっじゃあぴったりですね。\nB: はい。\nB: そうですねはい。\nA: はい。\nB: えっと見たことある顔もあるし。\nA: えっとはい。").strip()

In [29]:
system_msg = """
あなたは日本語の会話品質の厳密評価者です。出力は必ず有効なJSONのみ。説明や前置きは書かないでください。
重要：これは「音声会話の書き起こし」です。人間の会話には相づち（はい、ええ、うん等）、ためらい、言い直し、短い応答、重なり、言いよどみが自然に含まれます。
これらは原則として減点対象ではありません。意味の通る範囲なら自然さやターン運用で加点し得ます。
減点は、明確な意味破綻/無関連/機械的反復/会話の前進阻害などに限定します。
{
  "coherence": int,
  "naturalness": int,
  "relevance": int,
  "instruction_following": int,
  "turn_taking": int,
  "overall": int,
  "rationale": "短い根拠説明"
}
"""

user_msg = f"""
    "次の会話出力を評価してください。これは人間同士の自然な音声会話の書き起こしです。"
    "句読点や軽微な誤記は減点しないでください。相づちや短文応答は自然さとして許容します。\n\n"

    対話テキスト:
    \"\"\"{dialogue_text}\"\"\"""".strip()

chat = [
    {"role": "system", "content": system_msg},
    {"role": "user", "content": system_msg + user_msg},
]

In [30]:
inputs = tokenizer.apply_chat_template(
    chat, add_generation_prompt=True, tokenize=True, return_tensors="pt"
).to(model.device)

In [31]:
with torch.no_grad():
    output = model.generate(
        inputs,
        max_new_tokens=200,
        temperature=0.7,
        do_sample=(0.7 > 0.0),
        top_p=0.95,
        repetition_penalty=1.05,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )[0]

In [32]:
print(tokenizer.decode(output, skip_special_tokens=True).strip())

以下は、タスクを説明する指示です。要求を適切に満たす応答を書きなさい。

### 指示:

あなたは日本語の会話品質の厳密評価者です。出力は必ず有効なJSONのみ。説明や前置きは書かないでください。
重要：これは「音声会話の書き起こし」です。人間の会話には相づち（はい、ええ、うん等）、ためらい、言い直し、短い応答、重なり、言いよどみが自然に含まれます。
これらは原則として減点対象ではありません。意味の通る範囲なら自然さやターン運用で加点し得ます。
減点は、明確な意味破綻/無関連/機械的反復/会話の前進阻害などに限定します。
{
  "coherence": int,
  "naturalness": int,
  "relevance": int,
  "instruction_following": int,
  "turn_taking": int,
  "overall": int,
  "rationale": "短い根拠説明"
}
"次の会話出力を評価してください。これは人間同士の自然な音声会話の書き起こしです。"
    "句読点や軽微な誤記は減点しないでください。相づちや短文応答は自然さとして許容します。

"

    対話テキスト:
    """B: はい。
A: はい。
B: いろんな顔がありますね。
A: 私は1枚紙があって名前が書いてあります。
B: はいあっ構え書いてあるの。
A: 123456789。
B: はい。
A: 10人。
B: 12345678910人いますね。
A: あっじゃあぴったりですね。
B: はい。
B: そうですねはい。
A: はい。
B: えっと見たことある顔もあるし。
A: えっとはい。"""

### 応答:
{
  "coherence": 9,
  "naturalness": 8,
  "relevance": 8,
  "instruction_following": 9,
  "turn_taking": 9,
  "overall": 9,
  "rationale": "会話は一貫して理解可能であり、自然さも保たれています。各発言が適切にターン運用され、意味の通らない部分はありません。"
}


In [27]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained("llm-jp/llm-jp-3.1-13b-instruct4")
model = AutoModelForCausalLM.from_pretrained("llm-jp/llm-jp-3.1-13b-instruct4", device_map="auto", torch_dtype=torch.bfloat16)
chat = [
    {"role": "system", "content": "あなたは日本語の会話品質の厳密評価者です。出力は必ず有効なJSONのみ。説明や前置きは書かないでください。"},
    {"role": "user", "content": "自然言語処理とは何か"},
]
tokenized_input = tokenizer.apply_chat_template(chat, add_generation_prompt=True, tokenize=True, return_tensors="pt").to(model.device)
with torch.no_grad():
    output = model.generate(
        tokenized_input,
        max_new_tokens=100,
        do_sample=True,
        top_p=0.95,
        temperature=0.7,
        repetition_penalty=1.05,
    )[0]
print(tokenizer.decode(output))


Loading checkpoint shards: 100%|██████████| 6/6 [00:09<00:00,  1.52s/it]
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> 以下は、タスクを説明する指示です。要求を適切に満たす応答を書きなさい。

### 指示:
自然言語処理とは何か

### 応答:
自然言語処理（Natural Language Processing, NLP）とは、コンピュータが人間の言語を理解し、生成し、操作する技術のことを指します。この分野は、人工知能（AI）や機械学習（ML）と密接に関連しており、以下のような主要なタスクを含みます：

1. テキストの解析:
   - 形態素解析: 単語や文節に分割し、それぞれの品詞を特定する。
   - 構文解析: 文の構造を解析し、主語、


In [28]:
print(tokenizer.apply_chat_template(chat, add_generation_prompt=True, tokenize=False))

<s>以下は、タスクを説明する指示です。要求を適切に満たす応答を書きなさい。

### 指示:
自然言語処理とは何か

### 応答:

